In [2]:
%pylab inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import *
import collections
from collections import defaultdict
from __future__ import division
from collections import Counter
from scipy.stats.mstats import chisquare

Populating the interactive namespace from numpy and matplotlib


/home/mmoser/anaconda3/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['types', 'datetime', 'test', 'chisquare', 'unique', 'info']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"



# parse ASE read counts for analysis with MBASED: 

Take only first 8 columns of file as rest is empty

cut -f 1-8 ASE.counts > ASE.counts.short

## Synchronize the ASE.counts.short files:

In [5]:
ASErep1 = "/home/moser/RNA-seq/2015/ASE/ASE_countsF1Paxi_Pinf_limb/Peax162.RNA2015.25.gsnap.s.bam_23.01.2017.1mioSNP.ase.counts"
ASErep2 = "/home/moser/RNA-seq/2015/ASE/ASE_countsF1Paxi_Pinf_limb/Peax162.RNA2015.26.gsnap.s.bam_23.01.2017.1mioSNP.ase.counts"
ASErep3 = "/home/moser/RNA-seq/2015/ASE/ASE_countsF1Paxi_Pinf_limb/Peax162.RNA2015.27.gsnap.s.bam_23.01.2017.1mioSNP.ase.counts"

In [6]:
def loadASE(filename): 
    file_dict = {}
    with open(filename, "r") as f: 
        next(f)  #skip the first line
        count = 0
        for l in f:
            count += 1
            l = l.strip()
            l = l.split('\t')
            head = '_'.join(l[:5])
            rest = l[5:8] #is now set to include 3 replicates !!! has to be changed for more or less replicates         
            file_dict[head] = rest
            #print file_dict
           
    return file_dict 


In [9]:
%%time
#sync the file for each position

t1 = loadASE(ASErep1)
t2 = loadASE(ASErep2)
t3 = loadASE(ASErep3)

#python3 specific
allpos = set(list(t1.keys()) + list(t2.keys()) + list(t3.keys()))

display(len(t1), len(t2), len(t3))


display(len(allpos))

out = []

#making changes to t1 - t3... how?

e = 0
for i in allpos: 
    e += 1
    head = i.split('_')[0]
    pos = i.split('_')[1]
    if i in t1 or i in t2 or i in t3: 
        #if i not in defaultdict, defaultdict gets extended by item i with default value 0
        out.append([i, t1.get(i, ['0', '0', '0']), t2.get(i, ['0', '0', '0']), t3.get(i, ['0', '0', '0'])])
        
display(len(out))
#display(out)

1011975

984570

1013417

1189302

1189302

CPU times: user 19.9 s, sys: 396 ms, total: 20.3 s
Wall time: 20.1 s


In [10]:
print(out[1])

print('\t'.join(out[1][0].split('_')[0:2] + out[1][0].split('_')[3:5]))

['Peaxi162Scf00069_1524862_._A_G', ['2', '3', '5'], ['2', '4', '6'], ['3', '5', '8']]
Peaxi162Scf00069	1524862	A	G


In [12]:
combined_counts = []
with open('/home/moser/RNA-seq/2015/ASE/ASE_countsF1Paxi_Pinf_limb/F1axin_combined.1mioSNP.24.01.2017.counts', "w") as f: 
        e = 0
        for i in out: 
            e += 1
            #print i
            id_loc = '\t'.join(i[0].split('_')[0:2] + i[0].split('_')[3:5])

            counts = '\t'.join(['\t'.join(i[1]), '\t'.join(i[2]) ,'\t'.join(i[3])])
    
            f.write(id_loc +'\t'+ counts + '\n')
            #print id_loc, counts
            
            li = id_loc.split('\t')
            li.extend(counts.split('\t'))
            combined_counts.append(li)
            if e < 10: 
                print(id_loc, counts)
                print(li)
            

Peaxi162Scf00823	232616	A	T 0	9	9	0	5	5	1	4	5
['Peaxi162Scf00823', '232616', 'A', 'T', '0', '9', '9', '0', '5', '5', '1', '4', '5']
Peaxi162Scf00069	1524862	A	G 2	3	5	2	4	6	3	5	8
['Peaxi162Scf00069', '1524862', 'A', 'G', '2', '3', '5', '2', '4', '6', '3', '5', '8']
Peaxi162Scf00740	134785	A	G 8	4	12	6	3	9	9	7	16
['Peaxi162Scf00740', '134785', 'A', 'G', '8', '4', '12', '6', '3', '9', '9', '7', '16']
Peaxi162Scf00069	1977898	A	T 0	2	2	0	1	1	0	1	1
['Peaxi162Scf00069', '1977898', 'A', 'T', '0', '2', '2', '0', '1', '1', '0', '1', '1']
Peaxi162Scf00721	753310	G	C 0	1	1	0	1	1	0	0	0
['Peaxi162Scf00721', '753310', 'G', 'C', '0', '1', '1', '0', '1', '1', '0', '0', '0']
Peaxi162Scf00160	1783735	T	C 0	0	0	0	1	1	0	1	1
['Peaxi162Scf00160', '1783735', 'T', 'C', '0', '0', '0', '0', '1', '1', '0', '1', '1']
Peaxi162Scf00080	1326588	G	T 33	37	70	17	25	42	30	40	70
['Peaxi162Scf00080', '1326588', 'G', 'T', '33', '37', '70', '17', '25', '42', '30', '40', '70']
Peaxi162Scf00673	148428	G	G 1	0	1	0	0	0	1	0	1


In [13]:
#IMPORTANT list to use in the next step when looking up genes matching the posistions
combined_counts[0]

['Peaxi162Scf00823',
 '232616',
 'A',
 'T',
 '0',
 '9',
 '9',
 '0',
 '5',
 '5',
 '1',
 '4',
 '5']

## Assign gene positions to ASE counts: 

In [14]:
#load genome annotation of interest

#check for correct version of annotation

gene_list = []

with open("/home/moser/P.AXILLARIS/Peaxi162_annotation_v3.gene_models.RENAMED.gff") as gff:
    ei = 0
    for i in gff: 
        if i.startswith("##"): 
            continue
        ei += 1
        #if ei < 10: 
        #    print i
        i = i.strip()
        i = i.split("\t")
        if i[2] != "gene":
            continue
        #gene = []
        #gene.append(i[0])
        #gene.append(i[3:5])
        
        gene_name = i[8].split(";")[0][3:]
        
        gene_list.append([gene_name, i[0], int(i[3]), int(i[4])])
        
gene_df = pd.DataFrame(gene_list, columns=['gene', 'sequence', 'first', 'last'])
display(gene_df.shape)
display(gene_df.head(10))
display(gene_df.loc[gene_df["sequence"]=="Peaxi162Scf00578"])

(35816, 4)

,gene,sequence,first,last
0,Peaxi162Scf00000g00013,Peaxi162Scf00000,127648,129894
1,Peaxi162Scf00000g01005,Peaxi162Scf00000,135977,137407
2,Peaxi162Scf00000g00022,Peaxi162Scf00000,186434,191458
3,Peaxi162Scf00000g00029,Peaxi162Scf00000,192216,196484
4,Peaxi162Scf00000g02011,Peaxi162Scf00000,213081,215698
5,Peaxi162Scf00000t00021,Peaxi162Scf00000,261986,262059
6,Peaxi162Scf00000g04005,Peaxi162Scf00000,441521,442286
7,Peaxi162Scf00000g00052,Peaxi162Scf00000,532339,535455
8,Peaxi162Scf00000t00051,Peaxi162Scf00000,559444,559517
9,Peaxi162Scf00000g00062,Peaxi162Scf00000,651984,652568


,gene,sequence,first,last
25739,Peaxi162Scf00578g00014,Peaxi162Scf00578,157372,160237
25740,Peaxi162Scf00578g00027,Peaxi162Scf00578,211930,215818
25741,Peaxi162Scf00578g00317,Peaxi162Scf00578,301997,306283
25742,Peaxi162Scf00578g00032,Peaxi162Scf00578,353629,357094
25743,Peaxi162Scf00578g00316,Peaxi162Scf00578,358707,361905
25744,Peaxi162Scf00578g00523,Peaxi162Scf00578,518766,520026
25745,Peaxi162Scf00578g00525,Peaxi162Scf00578,573881,582434
25746,Peaxi162Scf00578g00055,Peaxi162Scf00578,524461,526467
25747,Peaxi162Scf00578g00526,Peaxi162Scf00578,523088,523994
25748,Peaxi162Scf00578g00527,Peaxi162Scf00578,540520,563495


In [15]:
%%time

#write to output

#what does positions look like?
c = 0
k = 0
with open('/home/moser/RNA-seq/2015/ASE/ASE_countsF1Paxi_Pinf_limb/F1axin_combined.1mioSNP.24.12.2016.v3162.counts', 'w') as outf: 
    for e, i in enumerate(combined_counts):
        #print e, i
    
    
        #only SNPs within gene annotations get printed to file
        result = gene_df.loc[(gene_df["sequence"] == i[0])&(gene_df["first"] <= int(i[1]))&(gene_df["last"] >= int(i[1]))]
        
        rest = 0
        
        #display(gene_df.loc[gene_df['gene'] == result])
        if result.shape[0] > 0:
            c += 1
            stringo = '\t'.join(i)
            res = '\t'.join([str(result.values[0][i]) for i in (0,2,3)])

            fin = res + '\t' + stringo + '\n'
            #print stringo, res
            outf.write(fin)
            if c % 10000 == 0:
                k += 1
                print(str(k) + ' * 10K processed...')
        
print(c)

#optimize for runtime!!!

#afterwards, sort with "$bedtools sort -i ...v3162.counts" to not mix up SNPs when treated in R

14601 * 10K processed...
29337 * 10K processed...
44122 * 10K processed...
58825 * 10K processed...
73479 * 10K processed...
88179 * 10K processed...
102837 * 10K processed...
117527 * 10K processed...
132219 * 10K processed...
146857 * 10K processed...
161514 * 10K processed...
176262 * 10K processed...
191047 * 10K processed...
205576 * 10K processed...
220293 * 10K processed...
234995 * 10K processed...
249566 * 10K processed...
264298 * 10K processed...
278985 * 10K processed...
293836 * 10K processed...
308502 * 10K processed...
323200 * 10K processed...
337953 * 10K processed...
352652 * 10K processed...
367323 * 10K processed...
381771 * 10K processed...
396498 * 10K processed...
411324 * 10K processed...
425911 * 10K processed...
440636 * 10K processed...
455294 * 10K processed...
469839 * 10K processed...
484636 * 10K processed...
499168 * 10K processed...
513695 * 10K processed...
528487 * 10K processed...
543171 * 10K processed...
557691 * 10K processed...
572433 * 10K proce